In [1]:
from orcapod.execution_engines import NativeRayAsyncEngine
import orcapod as op
import pyarrow as pa

In [2]:
input_stream = op.streams.ImmutableTableStream(
    pa.Table.from_pylist([{"id": i, "x": i * 2, "y": i * 3} for i in range(30)]),
    tag_columns=["id"],
)

In [ ]:
ray_engine = NativeRayAsyncEngine(
    "ray://raycluster-op-test-kuberay-head-svc.ray.svc.cluster.local:10001"
)

2025-08-05 07:53:27,293	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
2025-08-05 07:53:30,607	WARNING utils.py:1280 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.48.0
    Python: 3.13.5
This process on Ray Client was started with:
    Ray: 2.48.0
    Python: 3.13.3



(autoscaler +33s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +33s) Adding 5 node(s) of type workergroup.
(autoscaler +33s) Resized to 6 CPUs, 5 GPUs.
(autoscaler +33s) No available node types can fulfill resource requests {'CPU': 1.0}*19. Add suitable node types to this cluster to resolve this issue.
(autoscaler +39s) No available node types can fulfill resource requests {'CPU': 1.0}*9. Add suitable node types to this cluster to resolve this issue.
(autoscaler +54s) No available node types can fulfill resource requests {'CPU': 1.0}*12. Add suitable node types to this cluster to resolve this issue.
(autoscaler +1m20s) No available node types can fulfill resource requests {'CPU': 1.0}*8. Add suitable node types to this cluster to resolve this issue.


In [4]:
from time import sleep


@op.function_pod("sum")
def add_numbers(x: int, y: int) -> int:
    """
    A simple function that adds two numbers.
    """
    sleep(0.5)
    return x + y

Run first synchronously

In [5]:
result_stream1 = add_numbers(input_stream)
result_stream1.run()
result_stream1.as_df()

id,sum
i64,i64
0,0
1,5
2,10
3,15
4,20
…,…
25,125
26,130
27,135


Now let's run it asynchronously using the Ray engine

In [8]:
result_stream2 = add_numbers(input_stream)
result_stream2.run(ray_engine)
result_stream2.as_df()

id,sum
i64,i64
0,0
1,5
2,10
3,15
4,20
…,…
25,125
26,130
27,135


**NOTE**: Depending on the availability of nodes and how Ray was configured, you may *not* see any improvement in the running speed for the example above (it may even take longer due to overhead!). If you observe that you don't seem to be getting any speed up, please consult your Ray cluster administrator.